In [10]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import TensorBoard
from keras.utils import image_dataset_from_directory
import datetime

In [12]:
epochs = 10
batch_size = 32
img_height = 224
img_width = 224

In [13]:
data_dir = "Celebrity Faces Dataset"

dataset = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode= 'int',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    interpolation='bilinear',
)

Found 1800 files belonging to 17 classes.


In [14]:
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# Define the number of classes
num_classes = dataset.reduce(0, lambda x, _: x + 1).numpy()

In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Initialize TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model with TensorBoard callback
model.fit(dataset, epochs=epochs, callbacks=[tensorboard_callback])


Epoch 1/10
57/57 [==============================] - 24s 404ms/step - loss: 18.0273 - accuracy: 0.0728
Epoch 2/10
57/57 [==============================] - 24s 426ms/step - loss: 2.5667 - accuracy: 0.2550
Epoch 3/10
57/57 [==============================] - 23s 399ms/step - loss: 1.7196 - accuracy: 0.4844
Epoch 4/10
57/57 [==============================] - 25s 438ms/step - loss: 0.8535 - accuracy: 0.7456
Epoch 5/10
57/57 [==============================] - 25s 444ms/step - loss: 0.3643 - accuracy: 0.8894
Epoch 6/10
57/57 [==============================] - 26s 453ms/step - loss: 0.2214 - accuracy: 0.9422
Epoch 7/10
57/57 [==============================] - 24s 411ms/step - loss: 0.1893 - accuracy: 0.9461
Epoch 8/10
57/57 [==============================] - 23s 404ms/step - loss: 0.1661 - accuracy: 0.9567
Epoch 9/10
57/57 [==============================] - 26s 451ms/step - loss: 0.1127 - accuracy: 0.9739
Epoch 10/10
57/57 [==============================] - 26s 447ms/step - loss: 0.0258 - accur

In [ ]:
%tensorboard --logdir logs/fit

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import TensorBoard
import datetime

data_dir = 'Celebrity Faces Dataset'

img_width, img_height = 128, 128
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size,
          callbacks=[tensorboard_callback])

test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size, verbose=2)
print("\nTest Accuracy:", test_acc)



Found 1440 images belonging to 17 classes.
Found 360 images belonging to 17 classes.



Epoch 1/10


45/45 [==============================] - 19s 340ms/step - loss: 2.8303 - accuracy: 0.1035 - val_loss: 2.8163 - val_accuracy: 0.1136
Epoch 2/10
45/45 [==============================] - 13s 294ms/step - loss: 2.8092 - accuracy: 0.1118 - val_loss: 2.7858 - val_accuracy: 0.1136
Epoch 3/10
45/45 [==============================] - 13s 294ms/step - loss: 2.7710 - accuracy: 0.1243 - val_loss: 2.7167 - val_accuracy: 0.1420
Epoch 4/10
45/45 [==============================] - 14s 317ms/step - loss: 2.7071 - accuracy: 0.1382 - val_loss: 2.6774 - val_accuracy: 0.1335
Epoch 5/10
45/45 [==============================] - 14s 311ms/step - loss: 2.6410 - accuracy: 0.1604 - val_loss: 2.5759 - val_accuracy: 0.1733
Epoch 6/10
45/45 [==============================] - 13s 300ms/step - loss: 2.5981 - accuracy: 0.1653 - val_loss: 2.5250 - val_accuracy: 0.1761
Epoch 7/10
45/45 [==============================] -

In [2]:
%tensorboard --logdir logs/fit/

UsageError: Line magic function `%tensorboard` not found.
